In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

################################################################################
# --------------------- Utility functions --------------------------------------
################################################################################

def read_collision_to_df(filepath):
    temps = []
    data = []

    with open(filepath, 'r') as f:
        lines = f.readlines()

    # Find temperature header
    for i, line in enumerate(lines):
        if line.strip().startswith('!COLLISION TEMPERATURES'):
            temps_line = lines[i+1]
            temps = [float(v) for v in temps_line.strip().split()]
            break

    rate_columns = [f"{int(t)}Ke" for t in temps]
    all_columns = ['NUM', 'UP', 'LOW'] + rate_columns
    expected_length = len(all_columns)

    # Find start of transitions
    start_idx = None
    for i, line in enumerate(lines):
        if line.strip().startswith("!NUM"):
            start_idx = i + 1
            break

    # Only lines with expected number of values are parsed
    for line in lines[start_idx:]:
        parts = line.split()
        if len(parts) != expected_length:
            continue # skip anything that's not a full data row
        try:
            nums = list(map(int, parts[:3]))
            rates = list(map(float, parts[3:]))
            row = nums + rates
            data.append(row)
        except Exception:
            continue

    df = pd.DataFrame(data, columns=all_columns)
    df['nu'] = df['UP']
    df['nl'] = df['LOW']
    return df



def read_level_file(filepath, startline, columns, endline=None):
    """
    Reads a level file with columns [n,g,EnK,o,v,J,comment], returns a DataFrame.
    """
    lines = []
    with open(filepath, 'r') as f:
        for idx, line in enumerate(f):
            if idx >= startline and (endline is None or idx < endline):
                parts = line.strip().split()
                if len(parts) >= len(columns):
                    lines.append(parts[:len(columns)])
    df = pd.DataFrame(lines, columns=columns)
    for col in ['n', 'g', 'v', 'J']:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    return df

def merge_level_data_with_single_list(df_transitions, df_levels, up_col='nu', low_col='nl'):
    """
    For each transition, add quantum numbers v, J for both upper and lower levels
    from df_levels, matched by n (level index), using suffix 'u' for up and 'l' for low.
    up_col/low_col should be present in df_transitions.
    """
    df_levels_up = df_levels.rename(columns={'n': up_col, 'v': 'vu', 'J': 'Ju'})
    df_merged = pd.merge(df_transitions, df_levels_up[[up_col, 'vu', 'Ju']], on=up_col, how='left')
    df_levels_low = df_levels.rename(columns={'n': low_col, 'v': 'vl', 'J': 'Jl'})
    df_merged = pd.merge(df_merged, df_levels_low[[low_col, 'vl', 'Jl']], on=low_col, how='left')
    return df_merged

def rename_meudon_columns(df):
    """Rename Meudon columns to Stancil format, also creates nu/nl."""
    mapping = {'NUM': 'ID', 'UP': 'nu', 'LOW': 'nl'}
    for col in df.columns:
        if col.endswith('PDR'):
            mapping[col] = col.replace('KPDR', 'Ke')
    df = df.rename(columns=mapping)
    # Ensure nu and nl columns exist
    if 'UP' in df.columns: df['nu'] = df['UP']
    if 'LOW' in df.columns: df['nl'] = df['LOW']
    return df
    
def merge_out_pdr(df1, df2):
    """
    Merge df1 and unique rows from df2 (matched by ['vu','Ju','vl','Jl']),
    robust against duplicate columns, returns a valid concatenation.
    """
    keys = ['vu', 'Ju', 'vl', 'Jl']
    temp = pd.merge(df1, df2, on=keys, how='outer', indicator=True, suffixes=('_df1', '_df2'))

    # Only right_only (unique) rows from df2
    unique_df2 = temp[temp['_merge']=='right_only'].copy()
    # Rename _df2 columns
    rename_map = {col: col.replace('_df2', '') for col in unique_df2.columns if '_df2' in col}
    unique_df2.rename(columns=rename_map, inplace=True)
    # Drop duplicated columns by name (except keys)
    unique_df2 = unique_df2.loc[:, ~unique_df2.columns.duplicated()]

    # Ensure all columns from df1 exist (add missing)
    for col in df1.columns:
        if col not in unique_df2.columns:
            unique_df2[col] = np.nan
    # Order as df1
    unique_df2 = unique_df2[df1.columns]
    # Reset index just in case
    unique_df2.reset_index(drop=True, inplace=True)
    df1.reset_index(drop=True, inplace=True)
    merged = pd.concat([df1, unique_df2], ignore_index=True)
    return merged



def compare_dfs(df1, df2, keys, cols, epsilon=1):
    """
    Compare columns between two DFs, return mismatches above epsilon.
    Handles missing columns by filling with NaN so result is always robust.
    """
    mdf = pd.merge(df1, df2, on=keys, suffixes=('_df1', '_df2'))
    errors = {}
    for col in cols:
        c1, c2 = f"{col}_df1", f"{col}_df2"
        vals1 = mdf[c1] if c1 in mdf.columns else np.nan
        vals2 = mdf[c2] if c2 in mdf.columns else np.nan
        # If both are completely missing, skip this column
        if isinstance(vals1, float) and np.isnan(vals1) and isinstance(vals2, float) and np.isnan(vals2):
            print(f"Warning: Column {col} missing in both DataFrames after merge.")
            continue
        err = np.abs(vals1 - vals2)
        errors[f"{col}_error"] = err
    # Build mask (skip missing columns)
    mask_matrix = [v > epsilon for v in errors.values()]
    if mask_matrix:
        mask = np.column_stack(mask_matrix).any(axis=1)
        cols_to_return = [*keys] + sum(([f"{col}_df1", f"{col}_df2", f"{col}_error"] for col in cols if f"{col}_error" in errors), [])
        result = mdf.loc[mask, cols_to_return]
    else:
        result = pd.DataFrame(columns=[*keys])
    return result


################################################################################
# --------------------- Plotting ----------------------------------------------
################################################################################

def plot_transitions(df, Tvalues, idxs=None, label="Transitions"):
    # Use up to 3 rows, never out-of-bounds
    if idxs is None:
        idxs = range(min(len(df), 3))
    if len(df) == 0:
        print("Warning: No transitions to plot.")
        return
    for i in idxs:
        if i < len(df):  # robust to short DataFrames
            row = df.iloc[i]
            plt.plot(Tvalues, [row.get(f"{int(T)}Ke", np.nan) for T in Tvalues],
                     '--', label=f"{row.get('vu','?')}-{row.get('vl','?')} {row.get('Ju','?')}-{row.get('Jl','?')}")
    plt.grid(alpha=0.5)
    plt.xlabel("Temperature (K)")
    plt.ylabel("Collision rate (cm³ s⁻¹)")
    plt.xscale("log")
    plt.yscale("log")
    plt.legend()
    plt.tight_layout()
    plt.title(label)
    plt.show()

def plot_matrix_comparison(df_error, cols):
    error_cols = [f"{col}_error" for col in cols if f"{col}_error" in df_error.columns]
    if not error_cols:
        print("No error columns to plot for this study.")
        return
    data = df_error[error_cols].to_numpy()
    plt.imshow(data, cmap='viridis', aspect='auto', vmin=0, vmax=10)
    plt.colorbar(label="Absolute relative error")
    plt.xlabel("Temperatures K")
    plt.ylabel("Lines Index")
    plt.title("Comparison error")
    plt.tight_layout()
    plt.show()


################################################################################
# -------------------- Main Study Pipeline -------------------------------------
################################################################################

def full_study(label, collision_file, level_file, stancil_settings, temp_cols):
    print(f"\n##### {label} #####")

    # Load Meudon
    df_meudon = read_collision_to_df(collision_file)
    df_levelco = read_level_file(level_file, stancil_settings['level_start'], stancil_settings['level_columns'], stancil_settings['level_end'])
    df_meudon_full = merge_level_data_with_single_list(df_meudon, df_levelco, up_col='nu', low_col='nl')
    df_meudon_renamed = rename_meudon_columns(df_meudon_full)


    # Load Stancil
    df_stancil = read_collision_to_df(stancil_settings['stancil_file'])
    df_stancil_full = merge_level_data_with_single_list(df_stancil, df_levelco, up_col='nu', low_col='nl')

    # Merge Meudon & Stancil
    df_fusion = merge_out_pdr(df_stancil_full, df_meudon_renamed)
    df_fusion.sort_values(by=['vu','Ju','vl','Jl'], inplace=True)
    df_fusion.reset_index(drop=True, inplace=True)
    df_fusion['ID'] = df_fusion.index + 1

    plot_transitions(df_fusion, temp_cols, idxs=[0,1,2], label=f"{label}: New merged transitions")

    keys = ['vu','Ju','vl','Jl']
    comparison = compare_dfs(df_fusion, df_meudon_renamed, keys=keys, cols=temp_cols, epsilon=1)
    plot_matrix_comparison(comparison, temp_cols)

    return df_fusion, comparison

################################################################################
# ------------------- Example Usage -------------------------------------------
################################################################################

Tcols = ['2Ke', '3Ke', '5Ke', '7Ke', '10Ke', '15Ke', '20Ke', '30Ke', '50Ke', '70Ke', '100Ke', '150Ke', '200Ke', '300Ke', '500Ke', '700Ke', '1000Ke', '1500Ke', '2000Ke', '3000Ke', '5000Ke', '7000Ke', '10000Ke', '15000Ke', '20000Ke']

studies = {
    'H': {
        'collision_file': 'coll_co_h.dat',
        'level_file': 'level_co.dat',
        'stancil_file': 'co_Stancil2024.dat',
        'level_start': 7,
        'level_end': 253,
        'level_columns': ['n', 'g', 'EnK', 'o', 'v', 'J', 'comment'], },
    'oH2': {
        'collision_file': 'coll_co_oh2.dat',
        'level_file': 'level_co.dat',
        'stancil_file': 'co_Stancil2024.dat',
        'level_start': 7,
        'level_end': 253,
        'level_columns': ['n', 'g', 'EnK', 'o', 'v', 'J', 'comment'],  },
    'pH2': {
        'collision_file': 'coll_co_pH2.dat',
        'level_file': 'level_co.dat',
        'stancil_file': 'co_Stancil2024.dat',
        'level_start': 7,
        'level_end': 253,
        'level_columns': ['n', 'g', 'EnK', 'o', 'v', 'J', 'comment'], },
}

for label, params in studies.items():
    stancil_params = {
        'stancil_file': params['stancil_file'],
        'level_start': params['level_start'],
        'level_end': params['level_end'],
        'level_columns': params['level_columns'],
    }
    df_final, df_compare = full_study(
        label,
        params['collision_file'],
        params['level_file'],
        stancil_params,
        Tcols
    )



##### H #####


ValueError: invalid literal for int() with base 10: '2Ke'

NameError: name 'df_meudon' is not defined